In [1]:
!pip install faiss-cpu
!pip install -U sentence-transformers

     |████████████████████████████████| 8.2MB 14.0MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 2.2MB 25.9MB/s 
     |████████████████████████████████| 1.2MB 44.6MB/s 
     |████████████████████████████████| 870kB 28.8MB/s 
     |████████████████████████████████| 3.3MB 45.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=4cd59197c5b3a151238c32f9cc98b92a5c92ffa3484a725afe10545d0e1a45b0
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=84f6ea2ff11253059b7956c064bc7b2d59dc30519a60375706b5b72ea9ee847d
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [2]:
import numpy as np
import torch
import os
import pandas as pd
import faiss
import time
from sentence_transformers import SentenceTransformer

In [3]:
import pandas as pd

df = pd.read_csv("https://krono.act.uji.es/NLP-course/chitchat-friendly-es.csv")


In [4]:
questions = df['Question'].unique()
answers   = df['Answer'].unique()

print(len(questions),len(answers))

19112 88


In [5]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')


In [6]:
encoded_answers = model.encode(answers)

encoded_questions = model.encode(questions)

In [10]:
flatid = encoded_answers[0].shape[0]

In [11]:
index_Q = faiss.IndexIDMap(faiss.IndexFlatIP(flatid))
index_Q.add_with_ids(encoded_questions, np.array(range(0, len(encoded_questions))))

index_A = faiss.IndexIDMap(faiss.IndexFlatIP(flatid))
index_A.add_with_ids(encoded_answers, np.array(range(0, len(encoded_answers))))

In [12]:
def search(index, query, top=5):
   t=time.time()
   query_vector = model.encode([query])
   k = top
   top_k = index.search(query_vector, k)
   #print(query)
   #print('totaltime: {}'.format(time.time()-t), np.min(top_k[0]))
   return [top_k[1].tolist()[0], np.min(top_k[0])]

In [19]:
res = search(index_A, '¿Qué edad tienes?', top=1)
print('\n'.join([answers[i] for i in res[0]]))
print('Dist:',res[1])

En realidad, no tengo edad. Los bots no cumplimos años. 
Dist: 0.31637532


In [21]:
res = search(index_Q, '¿Cuántos años tienes?', top=4)
print('\n'.join([questions[i] for i in res[0]]))
print("Min Dist:", res[1])

¿Cuántos años tienes?
Que cuántos años tienes
Dime cuántos años tienes
¿Qué edad tienes?
Dist: 0.69604814


In [ ]:
## Ejercicio: evaluar cuántas respuestas acierta a las preguntas del xitxat